In [98]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import requests
import json

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, BaseMessage

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "tools"

In [99]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [100]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'Decide whether the task should be added to the ToDo list or to the calendar (if time is provided) and return the corresponding JSON', 'hint': 'always use YYYY-MM-DD format for dates', 'example for ToDo': 'Przypomnij mi, że mam kupić mleko = {"tool":"ToDo","desc":"Kup mleko" }', 'example for Calendar': 'Jutro mam spotkanie z Marianem = {"tool":"Calendar","desc":"Spotkanie z Marianem","date":"2024-04-10"}', 'question': 'Pojutrze mam wizytę u dentysty'}


In [105]:
type_schema = {
    "name": "decide_tool",
    "description": "Zdecyduj, czy zadanie powinno zostać dodane do listy ToDo, czy do kalendarza (jeśli podano czas) i zwróć odpowiedni JSON",
    "parameters": {
        "type": "object",
        "properties": {
            "ToDo": {
                "type": "string",
                "desc": "zadanie do zrobienia bez zaplanowanego terminu",
            },
            "Calendar": {
                "type": "string",
                "desc": "zadanie do wykonania w konkretnym terminie",
            }
        }
    },
    "required": ["name"]
}

In [106]:
def parse_function_call(result: BaseMessage):
    if result.additional_kwargs and "function_call" in result.additional_kwargs:
        return {
            "name": result.additional_kwargs["function_call"]["name"],
            "args": json.loads(result.additional_kwargs["function_call"]["arguments"]),
        }
    return None

In [107]:
model = ChatOpenAI(
    model = "gpt-4-0613",
).bind(functions = [type_schema])

result = model.invoke([
    HumanMessage(f"{response['question']}"),
])

action = parse_function_call(result)
print(action)

{'name': 'decide_tool', 'args': {'Calendar': 'Pojutrze wizyta u dentysty'}}


In [108]:
chat = ChatOpenAI(
    model = 'gpt-4'
)

if 'Calendar' in action['args']:

    calendar = chat.invoke([
        SystemMessage("""Dziś jest 2024-04-09. Zwróć datę, o której mowa w wiadomości i zapisz wiadomość w formie krótkiego zadania do zapisania w kalendarzu. 
                      ###
                      user: Jutro mam spotkanie z Marianem
                      AI: {"tool":"Calendar","desc":"Spotkanie z Marianem","date":"2024-04-10"}
                      ###
                      """),
        HumanMessage(f"Wiadomość: {response['question']}")
    ])

    answer = calendar.content

elif 'ToDo' in action['args']:
    todo = chat.invoke([
        SystemMessage("""Zapisz wiadomość w formie krótkiego zadania do zapisania w liście To Do. 
                      ###
                      user: Przypomnij mi, że mam kupić mleko 
                      AI: {"tool":"ToDo","desc":"Kup mleko"}
                      ###
                      """),
        HumanMessage(f"Wiadomość: {response['question']}")
    ])

    answer = todo.content


In [109]:
answer_json = json.loads(answer)

In [110]:
# wysłanie odpowiedzi

response_answ = send_answer(token, answer_json)
if response_answ:
    print("Odpowiedź z serwera:", response_answ)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


In [111]:
response['question']

'Pojutrze mam wizytę u dentysty'

In [114]:
answer_json

{'tool': 'Calendar', 'desc': 'Wizyta u dentysty', 'date': '2024-04-11'}